In [1]:
#__bojo90__

from __future__ import print_function
from fenics import *
import mshr
import numpy as np
%matplotlib inline

######################################################################################################################
class ChemicalReactionrate:
    """ 
    Chemical reaction of [NO], [0_3] and [NO_2]
    
    ===============     ===================================
    Attributes                       Description
    ===============     ===================================
    cloudCover          The ratio of sky obscured by cloud.
    solarElevation      The angle of the sun relative to 
                        the position of the plume.
    atmosphericTemp     Temperature of the ambient air.
    ===============     ===================================
        
    """
      
    def __init__(self, atmosphericTemp=274, cloudCover=0.5, solarElevation=0.79):
        """Initializes the Chemical reaction with given values for the atmospheric temperature, cloud cover
           and the solar elevation.
        """
        self.atmosphericTemp = atmosphericTemp
        self.cloudCover = cloudCover
        self.solarElevation = solarElevation
        
        
    def reactionRate(self):
        """ Reurns the ratio of the kinetic constants of reaction between [NO], [O_3] and [NO_2]. """ 
        k1 = 1/60*(0.5699-(9.056*(10**3)*(90-self.solarElevation))**2.546)*(1-0.75*(self.cloudCover/8)**3.4)
        k2 = 1.325*10**6*np.exp(-1430/self.atmosphericTemp)
        return k2/k1      


######################################################################################################################
class Domain:
    """ Two dimensional discretized domain for the plume transport. grid; defines the size of the domain.
        mesh; discretizes the domain. Function spaces are created for [NO], [O_3], [NO_2], [SO_2] and the 
        wind velocity, v.    
    """
       
    grid = mshr.Rectangle(Point(0, 0), Point(1.5, 1.5))
    mesh = mshr.generate_mesh(grid, 64)
    
    # Define function space for velocit
    velocityFunctionSpace = VectorFunctionSpace(mesh, 'P', 2)

    # Define function space for system of concentrations
    P1 = FiniteElement('P', triangle, 1)
    element = MixedElement(P1, P1, P1, P1)
    concentrationFunctionSpace = FunctionSpace(mesh, element)
    
    def velocityFunction(self):
        """Returns the velocity function for the wind velocity."""
        velocityFunction = Function(Domain.velocityFunctionSpace)
        return velocityFunction
        
    def concentrationFunction(self):
        """Returns concenteration function for [NO], [O_3], [NO_2], [SO_2]."""         
        concentrationFunction = Function(Domain.concentrationFunctionSpace)
        return concentrationFunction  
domain = Domain()
concentrationfunctioninitial = domain.concentrationFunction()
concentrationfunctionfinal = domain.concentrationFunction()
velocityfunction = domain.velocityFunction()

        
#####################################################################################################################    
class PlumeSource:
    """Plume source for [NO], [O_3] and [SO_2] are defined, and not for [NO_2] since it is produced
       as a result of the reaction between [NO] and [O_3]. 
    """
    
    def NO_Source(self):
        """[NO] source function for the plume transport."""
        self.NO_Source = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.2,2)<0.05*0.05 ? 0.1 : 0', degree=1)
        return self.NO_Source
        
    def O_3_Source(self):
        """[O_3] source function for the plume transport."""
        self.O_3_Source = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.3,2)<0.05*0.05 ? 0.1 : 0', degree=1)
        return self.O_3_Source
        
    def NO_2_Source(self):
        """[NO_2] source function for the plume transport."""
        self.NO_2_Source = Constant(0)
        return self.NO_2_Source

    def SO_2_Source(self):
        """[NO] source function for the plume transport."""
        self.SO_2_Source = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.4,2)<0.05*0.05 ? 0.1 : 0', degree=1)
        return self.SO_2_Source

       
#####################################################################################################################    
class VariationalSolver(PlumeSource):
    """Solver of the variational form of the four non-linear PDE's of [NO], [O_3], [NO_2]
       and [SO_2]. The total time of the simulation is defined, and also the time step. eps; represents 
       the diffusion coefficient of [[NO], [O_3], [NO_2] and [SO_2]. 
    """
    totalTime = 5.0    # final time
    num_steps = 500    # number of time steps
    timestep = totalTime / num_steps # time step size
    eps = 0.015         # diffusion coefficient

    ChemicalReaction = ChemicalReactionrate()
    reactionrate = ChemicalReaction.reactionRate()
    
    # Define expressions used in variational forms
    reactionrate = Constant(reactionrate)
    timeStep = Constant(timestep)
    eps = Constant(eps) 

    def solver(self):
        """Returns the solution to equation of the variational form from the combination of
           the PDE's of [NO], [O_3], [NO_2] and [SO_2].  
        """
        concentrationSplitInitialTime = split(concentrationfunctioninitial)
        concentrationSplitFinalTime = split(concentrationfunctionfinal)
        NO_testfunction, O_3_testfunction, NO_2_testfunction,  SO_2_testfunction = TestFunctions(
                                                                                domain.concentrationFunctionSpace)
        NO_initial, O_3_initial, NO_2_initial, SO_2_initial = concentrationSplitInitialTime      
        NO_final, O_3_final, NO_2_final, SO_2_final = concentrationSplitFinalTime        
        solver = ((NO_initial - NO_final) / VariationalSolver.timeStep)*NO_testfunction*dx \
                + dot(velocityfunction, grad(NO_initial))*NO_testfunction*dx \
                + VariationalSolver.eps*dot(grad(NO_initial), grad(NO_testfunction))*dx \
                + VariationalSolver.reactionrate*NO_initial*O_3_initial*NO_testfunction*dx \
                + ((O_3_initial - O_3_final) / VariationalSolver.timeStep)*O_3_testfunction*dx \
                + dot(velocityfunction, grad(O_3_initial))*O_3_testfunction*dx \
                + VariationalSolver.eps*dot(grad(O_3_initial), grad(O_3_testfunction))*dx \
                + VariationalSolver.reactionrate*NO_initial*O_3_initial*O_3_testfunction*dx \
                + ((NO_2_initial - NO_2_final) / VariationalSolver.timeStep)*NO_2_testfunction*dx \
                + dot(velocityfunction, grad(NO_2_initial))*NO_2_testfunction*dx \
                + VariationalSolver.eps*dot(grad(NO_2_initial), grad(NO_2_testfunction))*dx \
                - VariationalSolver.reactionrate*NO_initial*O_3_initial*NO_2_testfunction*dx \
                + VariationalSolver.reactionrate*NO_2_initial*NO_2_testfunction*dx \
                +((SO_2_initial - SO_2_final) / VariationalSolver.timeStep)*SO_2_testfunction*dx \
                + dot(velocityfunction, grad(SO_2_initial))*SO_2_testfunction*dx \
                + VariationalSolver.eps*dot(grad(SO_2_initial), grad(SO_2_testfunction))*dx \
                - self.NO_Source()*NO_testfunction*dx - self.O_3_Source()*O_3_testfunction*dx \
                - self.NO_2_Source()*NO_2_testfunction*dx - self.SO_2_Source()*SO_2_testfunction*dx

       
        return solver

######################################################################################################################    
def run_solver():   
    """Instantiates the variational solver. Creates vtk files where the data solution for the 
       transport and reaction of [NO], [O_3], [NO_2] and [SO_2] would be stored and later post-processed.
    """  
    variationalsolver = VariationalSolver()
    solver = variationalsolver.solver()
    num_steps = variationalsolver.num_steps
    timesteP = variationalsolver.timestep

    # Create time series for reading velocity data
    timeseries_velocityfunction = TimeSeries('wind-data/wind_data24')

    # Create VTK files for visualization output
    vtkfile_NO = File('reaction_system/NO.pvd')
    vtkfile_O_3 = File('reaction_system/O_3.pvd')
    vtkfile_NO_2 = File('reaction_system/NO_2.pvd')
    vtkfile_SO_2 = File('reaction_system/SO_2.pvd')

    # Time-stepping
    t = 0
    for n in range(num_steps):

        # Update current time
        t += timesteP

        # Read velocity from file
        try:
            timeseries_velocityfunction.retrieve(velocityfunction.vector(), t)
        except Exception as e:
            print(e)
            break

        # Solve variational problem for time step
        solve(solver == 0, concentrationfunctioninitial)
       
        # Save solution to file (VTK)
        _NO, _O_3, _NO_2, _SO_2 = concentrationfunctioninitial.split()
        vtkfile_NO << (_NO, t)
        vtkfile_O_3 << (_O_3, t)
        vtkfile_NO_2 << (_NO_2, t)
        vtkfile_SO_2 << (_SO_2, t)

        # Update previous solution
        concentrationfunctionfinal.assign(concentrationfunctioninitial)
        
if __name__ == '__main__':
    run_solver()